In [17]:
from langchain_google_genai import GoogleGenerativeAI as genai
from langchain_experimental.agents import create_csv_agent
from dotenv import get_key
import pandas as pd

In [27]:
df = pd.read_csv("train.csv",nrows=10)
df.head(10)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969
5,1000003,P00193542,M,26-35,15,A,3,0,1,2.0,NaN,15227
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
7,1000004,P00346142,M,46-50,7,B,2,1,1,15.0,NaN,15854
8,1000004,P0097242,M,46-50,7,B,2,1,1,16.0,NaN,15686
9,1000005,P00274942,M,26-35,20,A,1,1,8,NaN,NaN,7871


In [9]:
df.count()

User_ID                       10
Product_ID                    10
Gender                        10
Age                           10
Occupation                    10
City_Category                 10
Stay_In_Current_City_Years    10
Marital_Status                10
Product_Category_1            10
Product_Category_2             6
Product_Category_3             2
Purchase                      10
dtype: int64

In [12]:
api_key = get_key(".env", key_to_get="GEMINI_API_KEY")

In [13]:
llm = genai(model="gemini-1.5-flash-latest",google_api_key=api_key)

In [23]:
agent = create_csv_agent(llm=llm, path="train.csv",pandas_kwargs={"nrows" : 10},allow_dangerous_code=True, verbose = True)

In [24]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': '|    |   User_ID | Product_ID   | Gender   | Age   |   Occupation | City_Category   | Stay_In_Current_City_Years   |   Marital_Status |   Product_Category_1 |   Product_Category_2 |   Product_Category_3 |   Purchase |\n|---:|----------:|:-------------|:---------|:------|-------------:|:----------------|:-----------------------------|-----------------:|---------------------:|---------------------:|---------------------:|-----------:|\n|  0 |   1000001 | P00069042    | F        | 0-17  |           10 | A               | 2                            |                0 |                    3 |                  nan |                  nan |       8370 |\n|  1 |   1000001 | P00248942    | F        | 0-17  |           10 | A      

In [29]:
agent.run("what is the age group of males?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include males and then check the 'Age' column.
Action: python_repl_ast
Action Input: print(df[df['Gender'] == 'M']['Age'].unique())['55+' '26-35' '46-50']
Thought: I now know the age groups of males.
Final Answer: The age groups of males are '55+', '26-35', and '46-50'. 


> Finished chain.


"The age groups of males are '55+', '26-35', and '46-50'."

In [30]:
agent.run("what is the number of males?")



> Entering new AgentExecutor chain...
Thought: I can find the number of males by filtering the dataframe by gender and counting the rows.
Action: python_repl_ast
Action Input: print(len(df[df['Gender'] == 'M']))6
Thought: I now know the final answer
Final Answer: 6 


> Finished chain.


'6'